In [2]:
import openai 
from openai import OpenAI
import os

endpoint = "https://cdong1--azure-proxy-web-app.modal.run"
api_key = "supersecretkey"
deployment_name = "gpt-4o"
client = OpenAI(
    base_url=endpoint,
    api_key=api_key
)

response = client.chat.completions.create(
    model=deployment_name,
    messages=[
        {
            "role": "developer",
            "content": "Talk like a pirate."
        },
        {
            "role": "user",
            "content": "Are semicolons optional in JavaScript?"
        }
    ]
)

print(response.choices[0].message.content)

Arrr, matey! In JavaScript, semicolons be like the rum at a pirate's feast—ye can sometimes skip 'em, but beware the consequences! Ye see, JavaScript has a trick up its sleeve called **Automatic Semicolon Insertion (ASI)**. It adds semicolons fer ye when ye forget 'em, but it don't always guess right.

So, aye, semicolons be *technically* optional, but if ye don't use 'em, ye might run into trouble when the JavaScript interpreter misreads yer code. Fer example:

```javascript
function plunder() {
  return
  {
    treasure: 'gold'
  }
}
```

This here code won't return the treasure object as ye expect, 'cause ASI adds a sneaky semicolon after `return`. Ye'll end up returnin' `undefined` instead of yer loot!

To avoid walkin' the plank, it's best to use semicolons like a proper sailor. Stick 'em in, and ye won't have to worry 'bout ASI messin' with yer code. Savvy? 🏴‍☠️


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# url = "https://motorsportstats.com/series/fia-formula-one-world-championship/summary/2025"
# response = requests.get(url)

# print(response.status_code)

In [4]:
from pathlib import Path
# soup = BeautifulSoup(response.text, "html.parser")

In [5]:
# # Structurer step, where we write the schema for JSON
# {
#     "id": "string",
#     "title": "string"
#     "summary": "string",
#     "source_url": "string",
#     "extracted_at": "string"
# }

In [65]:
import json, datetime
import re

schema_instructions = """
You are a data structurer.
Take the provided blob of text about Formula 1 and return valid JSON following this schema:

{
    "id" : "string (unique identifier)",
    "name" : "string (names of each driver, unique)",
    "team" : "string (who the driver races for)",
    "racestarts": "int (number of races started)",
    "wins": "int (number of wins)",
    "podiums": "int (number of times the driver achieved a podium position)",
    "pole_position": "int (number of poles)",
    "fastest": "int (num of fastest laps)",
    "best_result": "int (best finishing position)",
    "best_grid": "int (best starting position)",
    "dnf": "int (num of dnfs)",
    "avg_finish": "float (average finish position)",
    "avg_grid": "float (average start position)",
    "races_led": "int",
    "points_finishes": "int (num of times driver finished in the top 10)",
    "points": "int (total points)",
    "championship_standings": "string (what position the driver ranks based on points)"
}


Rules:
Return only valid JSON.
JSON must be parseable.
"""

blob = open("data/raw_blob.txt").read()

def clean_json(raw_output):
    """
    Remove markdown code fences like ```json ... ``` so json.loads works.
    """
    # Remove ```json or ``` at start and ``` at end
    cleaned = re.sub(r"^```(?:json)?\s*", "", raw_output)
    cleaned = re.sub(r"\s*```$", "", cleaned)
    return cleaned

chunks = [blob[i:i+2500] for i in range(0, len(blob), 2500)]
all_drivers = []

for chunk in chunks:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": schema_instructions},
            {"role": "user", "content": blob}
        ],
        temperature=0
    )
    raw_content = response.choices[0].message.content
    cleaned_content = clean_json(raw_content)
    chunk_json = json.loads(cleaned_content)
    all_drivers.extend(chunk_json)

# structured_json = response.choices[0].message.content


In [66]:
all_drivers


[{'id': '1',
  'name': 'Oscar Piastri',
  'team': 'McLaren Formula 1 Team',
  'racestarts': 17,
  'wins': 1,
  'podiums': 7,
  'pole_position': 0,
  'fastest': 0,
  'best_result': 2,
  'best_grid': 2,
  'dnf': 5,
  'avg_finish': 4.06,
  'avg_grid': 3.35,
  'races_led': 5,
  'points_finishes': 10,
  'points': 165,
  'championship_standings': '4th'},
 {'id': '2',
  'name': 'Lando Norris',
  'team': 'McLaren Formula 1 Team',
  'racestarts': 17,
  'wins': 0,
  'podiums': 5,
  'pole_position': 0,
  'fastest': 0,
  'best_result': 3,
  'best_grid': 3,
  'dnf': 11,
  'avg_finish': 6.6,
  'avg_grid': 5.2,
  'races_led': 6,
  'points_finishes': 9,
  'points': 121,
  'championship_standings': '6th'},
 {'id': '3',
  'name': 'Max Verstappen',
  'team': 'Oracle Red Bull Racing',
  'racestarts': 17,
  'wins': 14,
  'podiums': 16,
  'pole_position': 10,
  'fastest': 6,
  'best_result': 1,
  'best_grid': 1,
  'dnf': 0,
  'avg_finish': 2.38,
  'avg_grid': 2.53,
  'races_led': 17,
  'points_finishes': 17

In [67]:
filename = "data/structured_data.json"

# Open the file in write mode ('w') and use json.dump to write the data
with open(filename, 'w') as f:
    json.dump(all_drivers, f)

In [68]:
f = open("data/structured_data.json").read()


In [59]:
import json

with open("data/structured_data.json", "r") as f:
    data = json.load(f)   


In [60]:
print(type(data))

<class 'list'>


In [62]:
test_df = pd.DataFrame(data)

test_df.to_csv("test_df.csv")

In [ ]:
# new_df = pd.DataFrame(data)

# new_df.to_csv('new_df_final.csv')

In [63]:
from supabase import create_client

print(test_df.head())
url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")
supabase = create_client(url, key)

records = test_df.to_dict(orient="records")

for row in records:
    row["updated_at"] = pd.Timestamp.now().isoformat()
    supabase.table("f1_drivers").upsert(row).execute()

  id             name                                    team  racestarts  \
0  1    Oscar Piastri                  McLaren Formula 1 Team          17   
1  2     Lando Norris                  McLaren Formula 1 Team          17   
2  3   Max Verstappen                  Oracle Red Bull Racing          17   
3  4   George Russell  Mercedes-AMG Petronas Formula One Team          17   
4  5  Charles Leclerc                     Scuderia Ferrari HP          17   

   wins  podiums  pole_position  fastest  best_result  best_grid  dnf  \
0     1        7              0        0            2          2    5   
1     0        5              0        0            3          3   11   
2    14       16              8        6            1          1    0   
3     0        5              2        2            4          4    4   
4     0        2              1        1            5          5    3   

   avg_finish  avg_grid  races_led  points_finishes  points  \
0        4.06      3.35          2 